In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import shutil

base_path = "/content/drive/MyDrive/CDAC Project/lora_output"

# Checkpoint folders from your training
checkpoint_folders = ["checkpoint-20", "checkpoint-40", "checkpoint-60", "checkpoint-80", "checkpoint-99", "checkpoint-100"]

# The model file we want from each folder
model_file_name = "model.safetensors"

# Loop to prepare each one
for i, ckpt_folder in enumerate(checkpoint_folders):
    name = f"checkpoint_{i+1}"  # New folder to store converted LoRA
    new_dir = os.path.join(base_path, name)
    os.makedirs(new_dir, exist_ok=True)

    # Source model file
    source_model_path = os.path.join(base_path, ckpt_folder, model_file_name)

    # Check if source model file exists
    if not os.path.exists(source_model_path):
        print(f"❌ Missing file: {source_model_path} — skipping")
        continue

    # Destination paths
    dest_model_path = os.path.join(new_dir, "adapter_model.safetensors")
    dest_config_path = os.path.join(new_dir, "adapter_config.json")

    # Copy and rename model file
    shutil.copy(source_model_path, dest_model_path)

    # Copy adapter_config.json from base path
    config_source = os.path.join(base_path, "adapter_config.json")
    if os.path.exists(config_source):
        shutil.copy(config_source, dest_config_path)
    else:
        print(f"❌ Missing adapter_config.json at {config_source}")
        continue

    print(f"✅ Created: {new_dir}")
    # Add file size check
    if os.path.exists(dest_model_path):
        file_size = os.path.getsize(dest_model_path)
        print(f"  📊 File size of {os.path.basename(dest_model_path)}: {file_size} bytes")
    else:
        print(f"  ⚠️ {os.path.basename(dest_model_path)} not found after copy.")


print("🎯 All checkpoints prepared.")

print("🎯 Done.")

✅ Created: /content/drive/MyDrive/CDAC Project/lora_output/checkpoint_1
  📊 File size of adapter_model.safetensors: 3440602632 bytes
✅ Created: /content/drive/MyDrive/CDAC Project/lora_output/checkpoint_2
  📊 File size of adapter_model.safetensors: 3440602632 bytes
✅ Created: /content/drive/MyDrive/CDAC Project/lora_output/checkpoint_3
  📊 File size of adapter_model.safetensors: 3440602632 bytes
✅ Created: /content/drive/MyDrive/CDAC Project/lora_output/checkpoint_4
  📊 File size of adapter_model.safetensors: 3440602632 bytes
✅ Created: /content/drive/MyDrive/CDAC Project/lora_output/checkpoint_5
  📊 File size of adapter_model.safetensors: 3440602632 bytes
✅ Created: /content/drive/MyDrive/CDAC Project/lora_output/checkpoint_6
  📊 File size of adapter_model.safetensors: 3455824250 bytes
🎯 All checkpoints prepared.
🎯 Done.


In [7]:
from diffusers import StableDiffusionPipeline
from peft import PeftModel
from PIL import Image
import torch
import os

# ✅ Define paths
base_path = "/content/drive/MyDrive/CDAC Project/lora_output"
checkpoint_dirs = [
    "checkpoint_1",
    "checkpoint_2",
    "checkpoint_3",
    "checkpoint_4",
    "checkpoint_5",
    "checkpoint_6"
]

# ✅ Define prompt to test
prompt = "A mother and a girl working in the kitchen, Studio Ghibli style"

# ✅ Load base model (Stable Diffusion v1.5)
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    safety_checker=None
).to("cuda")

# ✅ Loop through each LoRA checkpoint
for ckpt in checkpoint_dirs:
    lora_path = os.path.join(base_path, ckpt)
    output_path = os.path.join(base_path, f"output_{ckpt}.png")

    print(f"🔄 Loading LoRA from: {lora_path}")
    pipe.unet = PeftModel.from_pretrained(pipe.unet, lora_path).to("cuda")

    print(f"🎨 Generating image for: {prompt}")
    image = pipe(prompt).images[0]

    image.save(output_path)
    print(f"✅ Saved output to: {output_path}\n")


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


🔄 Loading LoRA from: /content/drive/MyDrive/CDAC Project/lora_output/checkpoint_1


/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:585: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.lora_A.default.weight', 'base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.lora_B.default.weight', 'base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_k.lora_A.default.weight', 'base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_k.lora_B.default.weight', 'base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_v.lora_A.default.weight', 'base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_v.lora_B.default.weight', 'base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_q.lora_A.default.weight', 'base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn2.to_q.lora_B.default.weight', 'base_model.model.down_blocks.0.attentions.0.transformer_

🎨 Generating image for: A mother and a girl working in the kitchen, Studio Ghibli style


...

  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved output to: /content/drive/MyDrive/CDAC Project/lora_output/output_checkpoint_5.png

🔄 Loading LoRA from: /content/drive/MyDrive/CDAC Project/lora_output/checkpoint_6
🎨 Generating image for: A mother and a girl working in the kitchen, Studio Ghibli style


/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:585: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.base_model.model.base_model.model.base_model.model.base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.lora_A.default.weight', 'base_model.model.base_model.model.base_model.model.base_model.model.base_model.model.base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_q.lora_B.default.weight', 'base_model.model.base_model.model.base_model.model.base_model.model.base_model.model.base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_k.lora_A.default.weight', 'base_model.model.base_model.model.base_model.model.base_model.model.base_model.model.base_model.model.down_blocks.0.attentions.0.transformer_blocks.0.attn1.to_k.lora_B.default.weight', 'base_model.model.base_model.model.base_model.model.base_model.model.base_model.model.base_model.model.down_blocks.0.atte

  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved output to: /content/drive/MyDrive/CDAC Project/lora_output/output_checkpoint_6.png



In [2]:
from diffusers import StableDiffusionPipeline
from peft import PeftModel
import torch
import os

# Define base path and checkpoint directories
base_path = "/content/drive/MyDrive/CDAC Project/lora_output"
checkpoint_dirs = [f"checkpoint_{i}" for i in range(1, 7)]
output_dir = "/content/drive/MyDrive/CDAC Project/output_images"
# Define multiple prompts
prompts = [
    "a cat and dog lying on mat ",
    "a man and boy sat on chair in garden",
    "a lion in the forest"]


# Load base pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    safety_checker=None
).to("cuda")

# Save original UNet to reset between adapters
base_unet = pipe.unet

# Loop through each checkpoint
for ckpt in checkpoint_dirs:
    lora_path = os.path.join(base_path, ckpt)

    print(f"\n🔄 Applying LoRA from: {lora_path}")
    pipe.unet = PeftModel.from_pretrained(base_unet, lora_path).to("cuda")

    # Loop through each prompt
    for i, prompt in enumerate(prompts, start=1):
        print(f"🎨 Generating image {i} for prompt: {prompt}")
        image = pipe(prompt).images[0]

        # Save output image
        output_path = os.path.join(output_dir, f"{ckpt}_prompt{i}.png")
        image.save(output_path)
        print(f"✅ Saved: {output_path}")


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

✅ Saved: /content/drive/MyDrive/CDAC Project/output_images/checkpoint_6_prompt1.png
🎨 Generating image 2 for prompt: a man and boy sat on chair in garden


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved: /content/drive/MyDrive/CDAC Project/output_images/checkpoint_6_prompt2.png
🎨 Generating image 3 for prompt: a lion in the forest


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved: /content/drive/MyDrive/CDAC Project/output_images/checkpoint_6_prompt3.png
